# Feature engineering

## Introduction
We always look at a round from the point of view of team_1.

## Features: 
- **target**: team_1 won the round
- **kills**: Amount of players killed by team_1
- **deaths**: Amount of players that died from team_1
- **equipment_value_ratio**: Total equipment value compared to equipment value of team_2. *This is better than saving the equipment values from both the teams. $650 in a pistol round is normal. $650 in round 10 can probably be considered as a saving round.*
- **first_kill**: Did team_1 make the first kill. Making the first kill results in a higher chance of winning the round.
- **map**: maybe there are more clutches on Mirage?
- **damage_done**: Amount of damage team1 did to team2
- **damage_taken**: Amount of damage team2 did to team1
- **median_player_health**: Median health of all players of the team.
- **is_bomb_planted**:

- **utility**: [TODO]

# Features to implement
- **defuse_kit_count**: Amount of defusekits.
  - **values**: [None,0,1,2,3,4,5] (None when team_1 are terrorists)
- **alive_ratio_on_bomb_planted**: Amount of players alive from team_1 compared to players alive from team_2 when the bomb is planted.
- **distance_from_bomb_on_plant**: Average distance from team_1 when bomb is planted. (Maybe they are already saving)




In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import sys
import os

import sys
sys.path.insert(0, '..')  # add parent folder path where lib folder is


In [2]:
# Feature engineering
from dp.database.connection import get_collections
import parser
from bson.objectid import ObjectId
from sklearn.model_selection import train_test_split

collections = get_collections()
Round = parser.Round.dataclass()

parsed_rounds = []
target = []
# TODO: remove this, we limit to 50 while testing.
matches = collections["matches"].find({}).limit(50)

# MATCHES
for match in matches:
  round_winstreak = 0
  team_1_id = match["teams"][0]

  # ROUND
  for index, round_id in enumerate(match["rounds"]):

    # Sometimes a round is not parsed correctly and the data is not available.
    round = collections["rounds"].find_one({ "_id": ObjectId(round_id)})
    if round is None:
      continue

    # Create a set of 4 ticks: @25%,@50%,@75%,@100%.
    # To learn the model it does not always has all kills and all information.
    duration = round["endTick"] - round["startTick"]
    ticks = list(map(lambda n: round["startTick"] + duration*n,[.35,.5,.75,1]))

    # We should only change this once every round. Not for the multiple tick points we check.
    round = parser.Round(round_id,team_1_id,round["endTick"], index)
    if round.is_win():
      round_winstreak+= 1
    else:
      round_winstreak=0

    for tick in ticks:
      round = parser.Round(round_id,team_1_id, tick)
      (kills, deaths) = round.kills_and_deaths()
      target.append(1) if round.is_win() else target.append(0)
      parsed_rounds.append(
        Round(
          kills=kills,
          deaths=deaths,
          first_blood=round.is_first_blood(),
          round_winstreak=round_winstreak,
          bomb_planted=round.is_bomb_planted(),
          is_terrorist=round.is_terrorist(),
          )
      )

print("Parsed rounds:", len(parsed_rounds))
data = pd.DataFrame(parsed_rounds)


ModuleNotFoundError: No module named 'dp'

In [ ]:
# Keep this separate otherwise the we can't rerun the cell above.
processed_data = parser.preprocessing(data)

In [ ]:
# We should how diverse our dataset is. If in 80% of our games team_1 wins.
# We will learn the model that it can always pick team_1 and get an accuracy of 80%. We don't want that :)

# Split the data in training and testing data
train_X,test_X, train_Y, test_Y = train_test_split(processed_data,target,test_size=0.2, random_state=42)
print(train_X)


      kills  deaths  round_winstreak  first_blood__0.0  first_blood__1.0
5054      2       4                1                 1                 0
120       0       0                0                 0                 0
2351      5       3                1                 1                 0
1907      5       4                1                 1                 0
3648      0       0                0                 0                 0
...     ...     ...              ...               ...               ...
3092      0       0                1                 0                 0
3772      0       0                0                 0                 0
5191      5       1                1                 0                 1
5226      2       1                3                 0                 1
860       0       0                0                 0                 0

[4275 rows x 5 columns]


# Hyperparameter tuning
With SVM we try to find the maximum margin separator between our two classes (team_1_wins, team_2_wins). This is the line the furtest from the nearest training data points. SVM calculates the distance to the closest datapoint for each possible line and picks the one with the highest distance. This makes an SVM a **maximum margin estimator. 

In most real probelms, it is not possible to find the perfect separating plane. Sometimes there are datapoints that are not closer to the other class. To handle this we allow the SVM to soften the margin. Which means we allow some of the points to creep into the margin if that allows a better fit. This transforms our SVM into a soft-margin classifier since we are allowing for a few mistakes. This is typically called C.

C is a hyperparameter that needs to be tuned based on the dataset.

In [ ]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV

parameters = {
  "kernel": ["rbf", "linear", "sigmoid"],
  "C": range(1,50),
}

clf = GridSearchCV(svm.SVC(probability=True), parameters)
clf.fit(train_X, train_Y)

print(clf.cv_results_)


# TODO: handle overfitting, with cross validation and folding

{'mean_fit_time': array([0.15326524, 0.00736089, 0.72939606, 0.10733724, 0.00652666,
       0.78051624, 0.08302202, 0.0056704 , 0.77337732, 0.08066244,
       0.0072484 , 0.79067626, 0.07577405, 0.00726919, 0.77460566,
       0.07531157, 0.00794411, 0.76167507, 0.07534509, 0.00711832,
       0.78074112, 0.07430444, 0.00740952, 0.76050344, 0.07004495,
       0.00811067, 0.76280937, 0.07295046, 0.0075758 , 0.73175282,
       0.07256269, 0.00622921, 0.73100939, 0.07500486, 0.00662436,
       0.75002508, 0.07342744, 0.00723772, 0.60656524, 0.07229605,
       0.00891185, 0.57188253, 0.07146854, 0.00766268, 0.4842876 ,
       0.08014736, 0.0077179 , 0.51942248, 0.06980186, 0.00541029,
       0.59644499, 0.06887851, 0.00660481, 0.57673736, 0.07330971,
       0.0071557 , 0.53611994, 0.06891394, 0.0065578 , 0.55244431,
       0.06851506, 0.00657372, 0.58906269, 0.0685976 , 0.00780292,
       0.58133097, 0.07351289, 0.00767207, 0.59569473, 0.06920228,
       0.00687475, 0.55692863, 0.07467103, 0

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
# Performance metrics
predictions = clf.predict(test_X)
print(predictions)
accuracy = accuracy_score(test_Y, predictions, normalize=True)
print(f"accuracy:{accuracy*100}%")
confusion_matrix(test_Y, predictions)



# TODO: Should add some more performance metrics to tweak the model.
# certainty for each category  (team1_wins, team1_loses)
print(clf.predict_proba(test_X))

[1 1 1 ... 1 1 0]
accuracy:100.0%
[[3.00000090e-14 1.00000000e+00]
 [3.00000090e-14 1.00000000e+00]
 [3.73673038e-07 9.99999626e-01]
 ...
 [3.00000090e-14 1.00000000e+00]
 [5.49284165e-06 9.99994507e-01]
 [9.99123614e-01 8.76386406e-04]]


In [ ]:
import pickle

# Export model to disk
try:
  filename = "../model/model.sav"
  pickle.dump(clf, open(filename, 'wb'))
except pickle.PickleError:
  print("Could not dump model.")


# Live demo parsing

A second part, we unfortunately had no time to implement, is **live demo parsing**.

We need to start by creating a probability N by N matrix, with N the different states.
This is also called a ** stochastic matrix**. Each combination of a row(x) and a column(y) is the probability to go from stateX to stateY.

Since we eventually want to find the winner of a game we also need something called absorbing states. **An absorbing state is a state that, once entered, cannot be left**: In this case ["team1_wins", "team2_wins"]
